# Landmark Recognition with Deep Belief Networks
-------------------------------------

<div style="text-align: right">Alfonso Irarrazaval · Gregory Schuit</div>
<div style="text-align: right">IIC3695 - Tópicos Avanzados de Inteligencia de Máquina</div>
<div style="text-align: right">Proyecto Semestral</div>
<div style="text-align: right">2019´1</div>


## 1. Introducción
-----------------
<div style="text-align: justify">
Alguna vez ha ocurrido que mientras revisas fotos de vacaciones pasadas, no puedes recordar exactamente que monumento o lugar histórico era ese para el que posaste tan felizmente? Para eso sirve el reconocimiento de Puntos de Referencia (Landmark Recognition de aquí en adelante), el que consiste precisamente en detectar que *Landmarks* están presentes en una foto [1].
</div>
<br>
<div style="text-align: justify">
Este es, en esencia, es un problema de reconocimiento de imagen, por lo que resulta intuitivo acudir a *Machine Learning* para encontrar su solución. El mayor obstaculo que enfrentan los problemas de este tipo es que hay demasiados *Landmarks* en que fijarse, por lo que se necesita un *dataset* muy grande para poder entrenar este modelo eficientemente. En este caso, ese detalle ha sido solucionado por Google®, ya que en su desafío presentado en Kaggle® [1] facilita el *dataset* mas grande de este tipo a la fecha.
</div>
<br>
<div style="text-align: justify">
El enfoque de este trabajo será centrado en *Bayesian Machine Learning*, específicamente en *Deep Belief Networks*, las cuales consideramos son un approach interesante para este tipo de problemas ya que permiten un aprendizaje modularizado de *features* (se detallará esto más adelante).
</div>
<br>
<div style="text-align: justify">
Se comenzará explicando la teoría detras de las *Deep Belief Networks*, detallando sus fundamentos probabilisticos y los elementos que las componen, para luego pasar al problema en cuestión y nuestra implementación de una posible solución.
</div>


## 2. Teoría
-----------------
### 2.1. Modelos Gráficos Probabilísticos
#### 2.1.1 Grafos dirigidos

<div style="text-align: justify">
&nbsp;&nbsp;&nbsp;&nbsp;Bajo el contexto de la teoría de probabilidades, existe un campo de estudio dedicado a representar dependencias entre eventos mediante grafos. Uno de los principales objetivos por los que nacen estos modelos gráficos es poder factorizar una distribución conjunta mediante probabilidades condicionales. Por ejemplo, la siguiente ecuación factoriza la probabilidad conjunta según el gráfico de la figura.
</div>

![image.png](img/bayesian_network.PNG)

<div style="text-align: center">$P(a, b, c) = P(a)P(b | a)P(c | a,b)  $ </div>


<div style="text-align: justify">
&nbsp;&nbsp;&nbsp;&nbsp;En general, la factorización guiada por una red de bayes se da según la siguiente fórmula:
</div>
<br>   
<div style="text-align: center">$P(x) = \prod_{k=1}^{K} P(x_{k}|pa_{k})$, </div>
<br>
<div style="text-align: justify">
donde $pa_{k}$ es el conjunto de todos los eventos que apuntan a $x_k$. Es importante notar que este tipo de grafos no puede poseer ciclos.
</div>
<br>  
<div style="text-align: justify">
&nbsp;&nbsp;&nbsp;&nbsp;Este tipo de factorización resulta muy útil al momento de generar muestras de una distribución compleja de varias variables. En el ejemplo anterior, el proceso para generar muestras de la conjunta es el siguiente: primero se genera una muestra de $a$ según su propia distribución, luego, dada la muestra de $a$, podemos generar una muestra de $b$ de manera directa, y finalmente, dada las muestras de $a$ y de $b$, condicionamos a $c$ y extraemos una muestra. El resultado final de la muestra son los tres valores extraídos en el proceso. A este proceso se le denomina _ancestral sampling_.
</div>
<br>            
         
          
<div style="text-align: justify">
&nbsp;&nbsp;&nbsp;&nbsp;En el contexto de modelos de inferencia, estos grafos pueden simplicar mucho el cómputo de predicciones, incluyendo nodos para parámetros, priors, hiperparámetros, etc. Un ejemplo rápido es la regresión lineal bayesiana que se muestra a continuación. Para más detalle, ver (Bishop, 2006).
</div>

<img src="img/regresion_bayes_network.PNG" width="250">

#### 2.1.2. Grafos no-dirigidos
<br>                 
<div style="text-align: justify">
&nbsp;&nbsp;&nbsp;&nbsp;Si bien las redes dirigidas descritas anteriormente simplifican de gran manera el modelamiento de problemas, podemos simplicar aún más las cosas para adecuarnos mejor a cierto tipo de problemas. Un problema de las redes dirigidas es que no se pueden modelar problemas sin generar ciclos (Koller & Friedman, Cap. 4.1, 2009). Además, el proceso de factorización dado un grafo extenso dirigido, que se realiza por medio de un algoritmo llamado _d-separation_, puede llegar a ser muy complejo (Bishop, 2006). En cambio, el proceso para determinar independencia entre grupos de nodos en un grafo no dirigido es mucho más simple. Considere la siguiente figura:</div>
<br>
<img src="img/red_no_dirigida.PNG" width="300">
<br>
<div style="text-align: justify">
&nbsp;&nbsp;&nbsp;&nbsp;Si queremos determinar si la relación $A \perp B\ |\ C\ $ (i.e. $A$ y $B$ son independientes dado $C$) se cumple, basta con mirar todos los caminos que van de $A$ a $B$, y ver si son bloquados por $C$. Si se cumple que todos los caminos pasan por $C$, se dice que $C$ bloquea los caminos, dejando a $A$ y $B$ independientes si se observa $C$. En caso de que exista al menos una conexión que no pase por $C$, la propiedad de independencia no necesariamente se cumple.
</div>
<br>
<div style="text-align: justify">
&nbsp;&nbsp;&nbsp;&nbsp;En este tipo de grafos, llamados _Markov fields_ o _Markov Networks_, cada arista posee un peso que puede interpretarse como una "afinidad" o "compatibilidad" entre los eventos que conecta. Es importante no confundir estos pesos con las probabilidades de los eventos o con las probabilidades condicionales. Estos pesos se denominan _factores_, y permiten factorizar una probabilidad conjunta de la siguiente forma:
</div>
<br>
<img src="img/misconception.PNG" width="150">
<br>
<div style="text-align: center">$P(A, B, C, D) = \phi(A, B) · \phi(B, C) ·\phi(C, D) ·\phi(A, D)$</div>
<br>
<div style="text-align: justify">
donde se logran plasmar relaciones de independencia entre $A$ y $C$ dados B y D, y entre $B$ y $D$ dados $A$ y $C$. Estas relaciones de independencia son imposibles de modelar mediante una red de bayes, tal y cómo se revisa en (Koller, Friedman, 2009), en donde se modela la situacion de 4 estudiantes ($A$, $B$, $C$ y $D$), que sólo se hablan entre los que sí poseen aristas, y que cada uno puede o no tener un mal entendido en la materia vista en clases. Este ejemplo es interesante, pero no se explicará porque sale del alcance de este informe.
</div>
<br>
<div style="text-align: justify">
&nbsp;&nbsp;&nbsp;&nbsp;El modelamiento de probabilidades conjuntas mediante una red de markov, es una generalización del método de factorización visto anteriormente para redes de Bayes (basta con definir $\phi$ igual a las probabilidades condicionales que correspondan), por lo que la capacidad de modelamiento de una red de Markov resulta ser superior. 
</div>
<br>
<div style="text-align: justify">
&nbsp;&nbsp;&nbsp;&nbsp;Formalmente, definimos la probabilidad conjunta de un vector $x$ como la _distribución de Gibbs_, correspondiente a una red de Markov, tal y como sigue:
</div>
<br>
<div style="text-align: center">$P(x_1, ..., x_n) = \phi(D_1) · \phi(D_2) ··· \phi(D_m) · (1/Z)$</div>
<br>
<div style="text-align: justify">
siendo Z la constante de normalización sobre todos los posibles x.
</div>
<br>
<div style="text-align: justify">
&nbsp;&nbsp;&nbsp;&nbsp;Una propiedad importante de esta factorización es que, si {$D_1$, ..., $D_m$} es un conjunto de cliques de la red, entonces la factorización puede levarse a cabo. De esto se desprende directamente que si estos cliques son máximos, estamos reduciendo lo más posible la cantidad de parámetros de la distribución factorizada.   
</div>

### 2.2. Boltzmann Machines

Las máquinas de Boltzmann, nacen como grafos no dirigidos (redes de markov), y constan, además de pesos en cada arista, a factores para cada uno de sus nodos, logrando así una capacidad aún mayor de modelamiento. De esta forma, definimos la suma de los logaritmos de los factores de la red como:
<br>
<img src="img/energy.PNG" width="400">
<br>

Esta función se denomina funcion de energía, y será la principal componente a optimizar en las RBM.

#### 2.2.2 Gibbs Sampling
Gibbs Sampling es un método ampliamente ocupado de Markov Chain Monte Carlo, el cual puede ser visto como un caso particular del algoritmo Metropolis-Hastings [2].

Consideramos la siguiente distribución de la que deseamos samplear, la cual tiene algun estado inicial para la MC:  
<div style="text-align: center">$p(\Theta) = p(\theta_{1}, \theta_{2}, ... , \theta_{m})$</div>  
Entonces, cada uno de los pasos de Gibbs consiste en reemplazar el valor de una de las variables $\theta_{i}$ por un valor sacado de la distribución de esa variable, condicionada por todas las demás variables del modelo
<div style="text-align: center">$p(\theta_{i}|\Theta - \{\theta_{i}\})$</div>  
Este proceso se repite reiteradas veces iterando sobre las variables.

En este caso, se utiliza Gibbs para simular la distribución de la *Markov Random Field* correspondiente. 
<div style="text-align: center">$MRF \rightarrow p(X) = p(X_{1}, X_{2}, ... , X_{m})$</div> 
En cada iteración de gibbs se actualizará el estado de la MRF a p(X^{t}) = p(X*{t}_{1}, X^{t}_{2}, ... , X^{t}_{m})$$




### 2.3 Deep Belief Networks (DBN's)
+ introducción a las DBN's

### 2.3 Image Recognition with DBN's

## 3. Problema
-----------------
Como se mencionó anteriormente, lo que se desea en este trabajo es desarrollar un algoritmo que se adhiera al principio de *Bayesian Machine Learning* para detectar correctamente si es que una foto presenta un landmark o no, y entregar el nombre de aquel landmark en caso de un resultado psitivo.

### 3.1 Descripción de datos

Los datos a utilizar corresponden al dataset expuesto en [1], pero dado que este dataset contiene >2.000.00 de imágenes y pesa >400GB, para efectos de simplicidad se utilizará un subset de éste que contiene todas las imágenes de los 14 *landmarks* con más imágenes presentes en el dataset (lo que de igual manera corresponde a 116.610 imágenes en 30.9 GB). El método de extracción de estas imagenes se detalla en el archivo downloader.py.  
La información se encuentra separada en 2 partes: 
+ Conjunto de imágenes en formato *.jpg* con landmarks presentes en ellas.
+ CSV relacionando nombre de archivo con landmark presente
En otras palabras, el archivo de imágenes corresponderá a el input de nuestro algoritmo, y del csv obtendremos el target respectivo. 

### 3.2 Approach propuesto


Una propuesta respecto al manejo de datos para la implementación, es comenzar usando un modelo que vectorice las imágenes, para luego buscar en esas imagenes patrones que representen a los distintos landmarks.

## 4. Conclusión

## 5. Referencias - Bibliografía

**[ 1 ]**  *Google Landmark Recognition Challenge* https://www.kaggle.com/c/landmark-recognition-challenge  
**[ 2 ]**  Falta poner todos los papers y los libros
**[ 3 ]**  
**[ 4 ]**  
**[ 5 ]**  
**[ 6 ]**  
**[ 7 ]**  
**[ 8 ]**  
**[ 9 ]**  
**[ 10 ]** 